In [ ]:
!pip install pandas
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -qU kiwipiepy konlpy langchain-teddynote tiktoken

In [42]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "dacon"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [43]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("dacon")

LangSmith 추적을 시작합니다.
[프로젝트명]
dacon


In [1]:
# for Retrieval
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever

# for llm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# for document loader
from langchain_community.document_loaders import PyMuPDFLoader

# for chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

# for embedding
from langchain_huggingface import HuggingFaceEmbeddings

# for vectorDB
from langchain_community.vectorstores import Chroma
from langchain.vectorstores import FAISS

# for chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain.schema import Document

# for huggingface
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from transformers import pipeline

# for data
import zipfile
import os
import unicodedata
import fitz

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

/Users/yuni/miniconda3/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"문서 {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [4]:
def format_docs(docs):
    unique_contents = set(doc.page_content for doc in docs)
    return "\n\n".join(unique_contents)

In [3]:
import pandas as pd
# 쿼리 처리
df = pd.read_csv("test.csv")

# Question 열의 텍스트를 리스트로 저장
questions = df['Question'].tolist()

In [5]:
# query test
query_test = questions[3]
query_test

'2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?'

In [6]:
### Document loader ###
pdfs = [
    "./test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf",
    "./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf",
    "./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf",
    "./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf",
    "./test_source/국토교통부_행복주택출자.pdf",
    "./test_source/보건복지부_노인장기요양보험 사업운영.pdf",
    "./test_source/보건복지부_부모급여(영아수당) 지원.pdf",
    "./test_source/산업통상자원부_에너지바우처.pdf",
    "./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf"
]

docs = [PyMuPDFLoader(pdf).load() for pdf in pdfs]
docs_list = [item for sublist in docs for item in sublist ]

# 총 데이터의 길이 출력
print("총 데이터 수:", len(docs_list))

In [11]:
### CSV loader ###
from langchain_community.document_loaders.csv_loader import CSVLoader
csvs = [
    "./test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.csv",
    "./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.csv",
    "./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.csv",
    "./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.csv",
    "./test_source/국토교통부_행복주택출자.csv",
    "./test_source/보건복지부_노인장기요양보험 사업운영.csv",
    "./test_source/보건복지부_부모급여(영아수당) 지원.csv",
    "./test_source/산업통상자원부_에너지바우처.csv",
    "./test_source/중소벤처기업부_혁신창업사업화자금(융자).csv"
]

# 데이터를 저장할 리스트
csv_data = []

# 각 CSV 파일을 로드하고 데이터를 리스트에 저장
for csv_file in csvs:
    loader = CSVLoader(file_path=csv_file, encoding='utf-8')
    data = loader.load()
    csv_data.extend(data)  # 각 파일의 데이터를 리스트에 추가

# 총 데이터의 길이 출력
print("총 데이터 수:", len(csv_data))

총 데이터 수: 494


In [ ]:
# PDF 데이터와 CSV 데이터를 결합
all_list = docs_list + csv_data
print("총 데이터 수:", len(all_list))

In [14]:
### Embedding ###
model_kwargs = {'device': 'cpu'} # gpu -> 'cuda', cpu -> 'cpu'

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs=model_kwargs,
    encode_kwargs=model_kwargs,
    show_progress=True
)

In [63]:
from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    #model="second-state/Mistral-Nemo-Instruct-2407-GGUF",
    model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    #model="Qwen/Qwen2-7B-Instruct-GGUF",
    #model= "teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
    temperature=0.0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # 스트림 출력 콜백
)

In [15]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [47]:
### csv + pdf ###

child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=50
)
# DB를 생성합니다.
vectorstore = Chroma(
    collection_name="full_documents", 
    embedding_function=embeddings
)
store = InMemoryStore()

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

retriever.add_documents(all_list, ids=None, add_to_docstore=True)

# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

Batches: 100%|██████████| 24/24 [01:44<00:00,  4.34s/it]


['3dc5f2dd-dbbd-4bef-a278-e4468d037d41',
 '10684ed8-53a5-4e30-a40a-95b177af5c3a',
 '297f4ec1-8d5c-430d-be5c-a8f88df8f6d6',
 '93b97984-a923-4b42-a0ee-b7b288f4d656',
 'c3337908-5f74-4dc1-acf3-48bb0bfb0cc5',
 '8cba15db-0fe5-47d2-8210-8a9173da222d',
 'a7cdf7e7-07b8-4b1b-96aa-71126e850890',
 '8728fdce-8846-4b4e-acfb-4c4f0e5b1b9a',
 'c0c9eae5-2d4c-4e29-9f11-30bce4d83272',
 '2076b74c-9922-45aa-9375-c3df5e300ae2',
 '0e8cb2db-7288-4bbe-9c70-2a43ea327438',
 'e3ef8565-99bc-49bf-965b-15eeb0aba9d5',
 '78d6ea11-7fca-4080-a3bf-83b4095769d1',
 '3ec70b8b-5e4e-4aba-816c-ca90cedefff1',
 'aeb9fd6c-c525-411f-ae9e-f983a7f9bff6',
 '8f0ecf35-6df6-4b3a-b360-a3c056110d80',
 'e55d089d-69f6-49db-8779-8ef30e86c25d',
 'bf600fff-3916-4e04-a01e-08cbfbbc8eee',
 'b8965b91-8fda-4034-a095-62e6b5f8f0cb',
 'a252e6f2-4e36-4122-b533-d30af7f46b96',
 'e188ef7b-4544-4d54-97a2-9aae26c9899f',
 'ea81b041-9679-43f6-81d7-00aa36106d53',
 'b78be4d6-e39e-4ea2-bfc7-82a44369e6fa',
 '69253649-d16b-4651-8abb-dccf2305872a',
 'ac5985d9-d7db-

In [ ]:
child_retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5, "fetch_k": 10}
)
query_test = questions[8]
print(query_test)
pretty_print_docs(child_retriever.invoke(query_test))

In [57]:
# kiwi retriever
from langchain_teddynote.retrievers import KiwiBM25Retriever
kiwi = KiwiBM25Retriever.from_documents(all_list)
kiwi.k = 5
print(query_test)
pretty_print_docs(kiwi.invoke(query_test))

혁신창업사업화자금(융자) 사업 집행절차는 어떻게 되나요?
문서 1:

구분: 명칭
기금: 창업환경조성
소관: 창업기업지원융자(기금)
실국(기관): 혁신창업사업화자금(융자)
계정: None
분야: None
부문: None
----------------------------------------------------------------------------------------------------
문서 2:

ㅇ 추진경위 
- 1998. 4
외환위기 이후 창업 활성화 및 고용 창출을 위하여 지원 시작
- 2005. 2
업체별 연간 대출한도를 20억원으로 확대(기존 10억원)
- 2006. 1
지원대상을 업력 5년미만까지 확대(기존 3년미만)
- 2008. 1
부채비율 예외적용 기업을 업력 5년미만까지 확대(기존 3년 미만)
- 2009. 1
지원대상을 업력 7년 미만까지 확대(기존 5년 미만)
- 2010. 3
재창업자금(실패 경영인에 대한 재기지원) 신규 지원
- 2012. 1
지원대상을 업력 5년 미만으로 축소(기존 7년 미만)
청년전용창업자금(만 39세 이하 청년창업자 대상) 신규 지원
- 2014. 2
경제혁신 3개년 계획에 반영(청년전용창업자금, 재창업자금)
- 2014. 1
지원대상 업력기준을 7년 미만으로 확대
- 2015. 1
재창업자금을 재도약지원자금(융자)의 내역사업으로 이관
- 2019. 1
청년전용창업자금을 혁신창업지원자금에 통합 운영
개발기술사업화자금을 내역사업으로 통합 및 일자리창출촉진자금 신규 지원
- 2020. 1
미래기술육성자금, 고성장촉진자금 신규 지원
- 2022. 1
미래기술육성자금 및 고성장촉진자금 사업 폐지
- 2023. 1
창업기반지원과 신청 대상이 중복인 일자리창출촉진자금을 폐지 
- 2023. 1
성장공유형 대출방식 추가 (혁신창업사업화자금)
< 국정과제 >
< 국정목표 2. 민간이 끌고 정부가 미는 역동적 경제 >
6. 중소·벤처기업이 경제의중심에 서는 나라
◾(31번) 중소기업 정책을 민간주도 혁신성장의 관점에서

In [58]:
### 2. Ensemble ###
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(
                    retrievers = [kiwi, child_retriever]
                    , weights = [0.2, 0.8],
                    search_type="mmr",
                    search_kwargs={'k':2})

In [64]:
### 3. reranker + ensemble ###
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

query_test = questions[0]

# 모델 초기화 
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=reranker, top_n=3)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke(query_test)

# 문서 출력
print(query_test)
pretty_print_docs(compressed_docs)

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.21s/it]


2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
문서 1:

사   업   명
혁신창업사업화자금(융자) (5152-301)
1. 사업 코드 정보
구분
기금
소관
실국(기관)
계정
분야
부문
코드
중소벤처기업
창
업
 및
 진
흥
기
금
중소벤처기업부중소기업정책실
110
118
명칭
산업·중소기업 
및 에너지
창업 및 벤처
구분
프로그램
단위사업
세부사업
코드
5100
5152
301
명칭
창업환경조성
창업기업지원융자(기금)
혁신창업사업화자금(융자)
2. 사업 지원 형태 및 지원율
직접
출자
출연
보조
융자
국고보조율(%) 융자율 (%)
○
○
3. 지출계획 총괄표
(단위: 백만원, %)
사업명
2022년
결산
2023년
2024년
증감
(B-A)
당초(A)
수정
확정(B)
(B-A)/
A
혁
신
창
업
사
업
화
자
금(융자)
2,300,000
2,230,000
2,330,000
2,007,800 △222,200
△9.96
4. 사업목적
ㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의
창업을 활성화하고 고용창출 도모
ㅇ (개발기술사업화) 중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의
제품화·사업화를 촉진하여 기술기반 중소기업을 육성
5. 사업근거 및 추진경위
ㅇ 법령상 근거 : 중소기업진흥에 관한 법률 제66조, 제67조, 제74조
중소기업창업지원법 제35조

----------------------------------------------------------------------------------------------------
문서 2:

사   업   명
혁신창업사업화자금(융자) (5152-301)
1. 사업 코드 정보
구분
기금
소관
실국(기관)
계정
분야
부문
코드
중소벤처기업
창
업
 및
 진
흥
기
금
중소벤처기업부중소기업정책실
110
118
명칭
산업·중소기업 
및 에너지
창업 및 벤처
구분
프로그램
단위사업
세부사업
코드
5100
5152
301
명칭
창업환경조성
창업기업지원융자(기

In [ ]:
# query test
query_test = questions[8]
print(query_test)
pretty_print_docs(compression_retriever.invoke(query_test))

In [67]:
# query test2
query_test = questions[2]
# 문서 내용을 저장할 변수 초기화
documents = compression_retriever.invoke(query_test)
documents_string = "" 

for document in documents:
    documents_string += f"""
----------------------------
{document.page_content}
"""
# Prompt    
prompt = PromptTemplate(
    template=""" 
    당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요. 
    문서의 내용을 자르거나 편집하지 않고 그대로 가져오세요.
    번호가 있는 경우 순서에 따른 번호를 매기지 마세요. 
    수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
    질문의 키워드를 바탕으로 문서를 끝까지 자세하고 꼼꼼하게 검토하세요. 
    
문서:
{document}

질문:
{query}

""",
    input_variables=["document", "query"] # 입력 변수 지정
)
print(query_test)

result = llm([
    HumanMessage(content=prompt.format(document=documents_string, query=query_test))
])

#print(result.content)

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.47s/it]


중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
중소기업진흥에 관한 법률 제66조, 제67조, 제74조와 중소기업창업지원법 제35조가 사업 근거입니다.

In [68]:
### 모든 질문에 대해 수행 ###
answers = []
for query in tqdm(questions, desc="Answer number"):
    documents = compression_retriever.invoke(query)
    
    # 문서 내용을 저장할 변수 초기화
    documents_string = ""

    for document in documents:
        documents_string += f"""
----------------------------
{document.page_content}
"""
    # Prompt
    prompt = PromptTemplate(
        template="""
            당신은 재정 정보 관련 전문가 입니다. 문서를 바탕으로 질문에 한 문장 이내로 답변하세요. 
            문서에 있는 내용을 자르거나 편집하지 않고 그대로 가져오세요.
            순서에 따른 번호를 매기지 마세요. 출력 시 불이익을 줄 것입니다.
            수치에 단위가 있다면 문서를 바탕으로 답변에 단위를 포함하세요.
            질문의 키워드를 바탕으로 문서를 끝까지 검토하세요.

            답변 외에 예시, 참고, 정보 출처, 신뢰도, 확장된 답변, '답변: ', '참고: '를 절대로 출력하지 마세요. 
        
            문서:
            {document},

            질문:
            {query},
            
            주어진 질문에 대한 답변만 한 문장으로 생성한다.    
            """,
        input_variables=["document", "query"] 
    )

    print(f"Question: {query}")

    chain = (
        {"document": compression_retriever | format_docs, "query": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )

    result = chain.invoke(query)
    answers.append(result)
    #print(f"Answer: {result}\n")

submission_df = pd.read_csv("sample_submission.csv")

submission_df['Answer'] = answers

# 수정된 데이터프레임을 다시 CSV 파일로 저장
submission_df.to_csv("submission.csv", index=False)

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Batches: 100%|██████████| 1/1 [00:03<00:00,  3.09s/it]


2022년 혁신창업사업화자금(융자의) 예산은 2,300,000 백만원입니다.

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.64s/it]6.34s/it]


...

Batches: 100%|██████████| 1/1 [00:06<00:00,  6.22s/it]


우리나라는 2011년부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있으나, 용어 사용에 혼란이 있고 분류상 오류가 있는 등 체계적 관리 대안에 대한 논의가 필요한 시점이다.

Batches: 100%|██████████| 1/1 [00:04<00:00,  4.32s/it], 785.96s/it]


Question: 우발부채 관련 주요 쟁점은 무엇인가?


Batches: 100%|██████████| 1/1 [00:05<00:00,  5.71s/it]
